In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import * 

In [0]:
df = spark.table("workspace.bronze.crm_sales_details")

#Silver Transformations

##Trimming

In [0]:
for field in df.schema.fields:
  if isinstance(field.dataType,StringType):
      df = df.withColumn(field.name,trim(col(field.name)))

In [0]:
df.display()

In [0]:
df = df.withColumn(
    "sls_order_dt",
    when(
        (col("sls_order_dt") == 0) | (length(col("sls_order_dt")) != 8),
        None
    ).otherwise(
        to_date(col("sls_order_dt").cast("string"), "yyyyMMdd")
    )
)

df = df.withColumn(
    "sls_ship_dt",
    when(
        (col("sls_ship_dt") == 0) | (length(col("sls_ship_dt")) != 8),
        None
    ).otherwise(
        to_date(col("sls_ship_dt").cast("string"), "yyyyMMdd")
    )
)

df = df.withColumn(
    "sls_due_dt",
    when(
        (col("sls_due_dt") == 0) | (length(col("sls_due_dt")) != 8),
        None
    ).otherwise(
        to_date(col("sls_due_dt").cast("string"), "yyyyMMdd")
    )
)

In [0]:
df.display()

#Sales and Price Corrections

In [0]:
df = df.withColumn(
    "sls_price",
    when(
        (col("sls_price").isNull() | (col("sls_price") <= 0)) &
        (col("sls_quantity") != 0),
        col("sls_sales") / col("sls_quantity")
    ).otherwise(col("sls_price"))
)

In [0]:
df.display()

In [0]:
RENAME_MAP = {
    "sls_ord_num": "order_number",
    "sls_prd_key": "product_number",
    "sls_cust_id": "customer_id",
    "sls_order_dt": "order_date",
    "sls_ship_dt": "ship_date",
    "sls_due_dt": "due_date",
    "sls_sales": "sales_amount",
    "sls_quantity": "quantity",
    "sls_price": "price"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.crm_sales")

In [0]:
%sql
SELECT * FROM workspace.silver.crm_sales LIMIT 10